In [1]:
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
pd.set_option('display.max_columns', None)

In [2]:
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

In [3]:
credit_card_balance=pd.read_csv(r'D:\kaggle\home credit risk\DATA\credit_card_balance.csv')


In [5]:
credit_card_balance.isnull().sum()/credit_card_balance.shape[0]

SK_ID_PREV                    0.000000
SK_ID_CURR                    0.000000
MONTHS_BALANCE                0.000000
AMT_BALANCE                   0.000000
AMT_CREDIT_LIMIT_ACTUAL       0.000000
AMT_DRAWINGS_ATM_CURRENT      0.195249
AMT_DRAWINGS_CURRENT          0.000000
AMT_DRAWINGS_OTHER_CURRENT    0.195249
AMT_DRAWINGS_POS_CURRENT      0.195249
AMT_INST_MIN_REGULARITY       0.079482
AMT_PAYMENT_CURRENT           0.199981
AMT_PAYMENT_TOTAL_CURRENT     0.000000
AMT_RECEIVABLE_PRINCIPAL      0.000000
AMT_RECIVABLE                 0.000000
AMT_TOTAL_RECEIVABLE          0.000000
CNT_DRAWINGS_ATM_CURRENT      0.195249
CNT_DRAWINGS_CURRENT          0.000000
CNT_DRAWINGS_OTHER_CURRENT    0.195249
CNT_DRAWINGS_POS_CURRENT      0.195249
CNT_INSTALMENT_MATURE_CUM     0.079482
NAME_CONTRACT_STATUS          0.000000
SK_DPD                        0.000000
SK_DPD_DEF                    0.000000
dtype: float64

In [6]:
credit_card_balance.fillna(0,inplace=True)

In [7]:
# na filled
credit_card_balance

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.500,0.0,877.500,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.000,0.0,0.000,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.000,0.0,0.000,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.000,0.0,0.000,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.000,0.0,11547.000,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0
5,2646502,380010,-7,82903.815,270000,0.0,0.000,0.0,0.000,4449.105,...,82773.315,82773.315,0.0,0,0.0,0.0,2.0,Active,7,0
6,1079071,171320,-6,353451.645,585000,67500.0,67500.000,0.0,0.000,14684.175,...,351881.145,351881.145,1.0,1,0.0,0.0,6.0,Active,0,0
7,2095912,118650,-7,47962.125,45000,45000.0,45000.000,0.0,0.000,0.000,...,47962.125,47962.125,1.0,1,0.0,0.0,51.0,Active,0,0
8,2181852,367360,-4,291543.075,292500,90000.0,289339.425,0.0,199339.425,130.500,...,286831.575,286831.575,3.0,8,0.0,5.0,3.0,Active,0,0
9,1235299,203885,-5,201261.195,225000,76500.0,111026.700,0.0,34526.700,6338.340,...,197224.695,197224.695,3.0,9,0.0,6.0,38.0,Active,0,0


In [20]:
credit_card_balance.shape

(3840312, 23)

In [22]:
credit_card_balance[credit_card_balance['SK_DPD']>0].iloc[:,:-4]

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT
5,2646502,380010,-7,82903.815,270000,0.0,0.000,0.0,0.000,4449.105,3825.000,3825.000,80519.040,82773.315,82773.315,0.0,0,0.0,0.0
215,2105237,354676,-4,48.825,45000,0.0,0.000,0.0,0.000,0.000,0.000,0.000,0.000,48.825,48.825,0.0,0,0.0,0.0
379,2720092,130276,-5,106638.525,112500,0.0,2971.485,0.0,2971.485,7875.000,7650.000,7650.000,103836.105,106638.525,106638.525,0.0,2,0.0,2.0
478,1376802,104523,-1,11866.815,135000,0.0,11866.995,0.0,11866.995,2250.000,0.000,0.000,11866.815,11866.815,11866.815,0.0,1,0.0,1.0
517,1489084,207482,-4,146687.445,225000,0.0,39060.000,0.0,39060.000,5481.675,4500.000,4500.000,139465.800,146687.445,146687.445,0.0,1,0.0,1.0
527,2812788,434122,-7,474823.215,450000,0.0,0.000,0.0,0.000,23184.225,0.000,0.000,448049.565,474692.715,474692.715,0.0,0,0.0,0.0
586,1736160,238108,-5,463702.905,450000,0.0,0.000,0.0,0.000,22645.620,0.000,0.000,434045.250,460360.755,460360.755,0.0,0,0.0,0.0
797,1003474,310842,-3,130.500,0,0.0,0.000,0.0,0.000,130.500,0.000,0.000,0.000,130.500,130.500,0.0,0,0.0,0.0
798,2505040,310598,-7,130.500,0,0.0,0.000,0.0,0.000,130.500,0.000,0.000,0.000,130.500,130.500,0.0,0,0.0,0.0
874,1476467,210762,-4,186501.375,180000,0.0,2888.550,0.0,2888.550,9323.550,9000.000,9000.000,179708.805,185768.370,185768.370,0.0,3,0.0,3.0


In [17]:
credit_card_balance.sort_values(by=['SK_ID_CURR','SK_ID_PREV','MONTHS_BALANCE'],axis=0).iloc[:,:-4]

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT
1636141,1489396,100006,-6,0.000,270000,NaN,0.0,NaN,NaN,0.000,NaN,0.000,0.000,0.000,0.000,NaN,0,NaN,NaN
655566,1489396,100006,-5,0.000,270000,NaN,0.0,NaN,NaN,0.000,NaN,0.000,0.000,0.000,0.000,NaN,0,NaN,NaN
1399895,1489396,100006,-4,0.000,270000,NaN,0.0,NaN,NaN,0.000,NaN,0.000,0.000,0.000,0.000,NaN,0,NaN,NaN
1347528,1489396,100006,-3,0.000,270000,NaN,0.0,NaN,NaN,0.000,NaN,0.000,0.000,0.000,0.000,NaN,0,NaN,NaN
520387,1489396,100006,-2,0.000,270000,NaN,0.0,NaN,NaN,0.000,NaN,0.000,0.000,0.000,0.000,NaN,0,NaN,NaN
584804,1489396,100006,-1,0.000,270000,NaN,0.0,NaN,NaN,0.000,NaN,0.000,0.000,0.000,0.000,NaN,0,NaN,NaN
3131464,1843384,100011,-75,189000.000,180000,180000.0,180000.0,0.0,0.0,NaN,0.0,0.000,180000.000,189000.000,189000.000,4.0,4,0.0,0.0
2447092,1843384,100011,-74,184568.850,180000,0.0,0.0,0.0,0.0,9000.000,9000.0,9000.000,180000.000,184568.850,184568.850,0.0,0,0.0,0.0
2353190,1843384,100011,-73,181044.540,180000,0.0,0.0,0.0,0.0,9000.000,9000.0,9000.000,175568.850,181044.540,181044.540,0.0,0,0.0,0.0
1086495,1843384,100011,-72,177544.350,180000,0.0,0.0,0.0,0.0,9000.000,9000.0,9000.000,172044.540,177544.350,177544.350,0.0,0,0.0,0.0


In [11]:
credit_card_balance,ccb_new_col=one_hot_encoder(credit_card_balance)

In [12]:
credit_card_balance

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,SK_DPD,SK_DPD_DEF,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Refused,NAME_CONTRACT_STATUS_Sent proposal,NAME_CONTRACT_STATUS_Signed,NAME_CONTRACT_STATUS_nan
0,2562384,378907,-6,56.970,135000,0.0,877.500,0.0,877.500,1700.325,...,0,0,1,0,0,0,0,0,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.000,0.0,0.000,2250.000,...,0,0,1,0,0,0,0,0,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.000,0.0,0.000,2250.000,...,0,0,1,0,0,0,0,0,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.000,0.0,0.000,11795.760,...,0,0,1,0,0,0,0,0,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.000,0.0,11547.000,22924.890,...,0,0,1,0,0,0,0,0,0,0
5,2646502,380010,-7,82903.815,270000,0.0,0.000,0.0,0.000,4449.105,...,7,0,1,0,0,0,0,0,0,0
6,1079071,171320,-6,353451.645,585000,67500.0,67500.000,0.0,0.000,14684.175,...,0,0,1,0,0,0,0,0,0,0
7,2095912,118650,-7,47962.125,45000,45000.0,45000.000,0.0,0.000,0.000,...,0,0,1,0,0,0,0,0,0,0
8,2181852,367360,-4,291543.075,292500,90000.0,289339.425,0.0,199339.425,130.500,...,0,0,1,0,0,0,0,0,0,0
9,1235299,203885,-5,201261.195,225000,76500.0,111026.700,0.0,34526.700,6338.340,...,0,0,1,0,0,0,0,0,0,0


In [80]:
def avg_amount_per_day(day,amount):
    if day >0:
        return(amount/day)
    else:
        return(0)
        

In [75]:
count_0=['AMT_DRAWINGS_ATM_CURRENT','AMT_DRAWINGS_CURRENT','AMT_DRAWINGS_OTHER_CURRENT','AMT_DRAWINGS_POS_CURRENT']
def ccb_f(col):
    length=col.shape[0]
    # number of total terms
    g=col[['SK_ID_PREV','AMT_INST_MIN_REGULARITY']].groupby(['SK_ID_PREV'])['AMT_INST_MIN_REGULARITY'].mean()
    
    return(pd.Series({'% of AMT_DRAWINGS_ATM_CURRENT':sum(col['AMT_DRAWINGS_ATM_CURRENT']>0)/length,
                      '% of AMT_DRAWINGS_CURRENT':sum(col['AMT_DRAWINGS_CURRENT']>0)/length,
                      '% of AMT_DRAWINGS_OTHER_CURRENT':sum(col['AMT_DRAWINGS_OTHER_CURRENT']>0)/length,
                      '% of AMT_DRAWINGS_POS_CURRENT':sum(col['AMT_DRAWINGS_POS_CURRENT']>0)/length,
                     'AVG monthly # of drawing':sum(col['CNT_DRAWINGS_ATM_CURRENT']+col['CNT_DRAWINGS_CURRENT']+col['CNT_DRAWINGS_OTHER_CURRENT']+col['CNT_DRAWINGS_POS_CURRENT'])/length,
                     '% of DPD':sum(col['SK_DPD'])/30/length,
                     '% of # of DPD':sum(col['SK_DPD']>0)/length,
                     'current debt':sum(col[col['MONTHS_BALANCE']==-1]['AMT_TOTAL_RECEIVABLE']),
                     'AVG due amount per due day':avg_amount_per_day(sum(col['SK_DPD']),sum(col['AMT_INST_MIN_REGULARITY']-col['AMT_PAYMENT_CURRENT'])),
                     'AVG untolerence due amount per due day':avg_amount_per_day(sum(col['SK_DPD_DEF']),sum(col['AMT_INST_MIN_REGULARITY']-col['AMT_PAYMENT_CURRENT'])),
                     '% of regular installment loan':sum(g[g>0]/len(g))}))
    
    

In [64]:
credit_card_balance.iloc[:5]

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [86]:
print(time.localtime())
credit_card_balance.groupby('SK_ID_CURR').apply(lambda col:ccb_f(col))
print(time.localtime())

time.struct_time(tm_year=2018, tm_mon=8, tm_mday=12, tm_hour=19, tm_min=24, tm_sec=30, tm_wday=6, tm_yday=224, tm_isdst=0)
time.struct_time(tm_year=2018, tm_mon=8, tm_mday=12, tm_hour=19, tm_min=29, tm_sec=30, tm_wday=6, tm_yday=224, tm_isdst=0)


In [ ]:
def credit_card_balance(num_rows = None, nan_as_category = True):
    gc.diable()
    cc = pd.read_csv('../input/credit_card_balance.csv', nrows = num_rows)
    cc.fillna(0,inplace=True)
    cc.drop(['SK_ID_PREV'], axis= 1, inplace = True)
    
    #custome aggregations
    g=cc.groupby(['SK_ID_CURR']).apply(lambda col:ccb_f(col))
    
    cc, cat_cols = one_hot_encoder(cc, nan_as_category= True)
    # General aggregations
    
    
    cc_agg = cc.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])
    cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])
    # Count credit card lines
    cc_agg['CC_COUNT'] = cc.groupby('SK_ID_CURR').size()
    cc_agg=pd.merge(cc_agg,g,on=['SK_ID_CURR'])
    gc.enable()
    del cc,g
    gc.collect()
    return cc_agg

## previous application

In [3]:
previous_application=pd.read_csv(r'D:\kaggle\home credit risk\DATA\previous_application.csv')

In [10]:
previous_application

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.000000,0.182832,0.867336,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,Y,1,NaN,NaN,NaN,XNA,Approved,-512,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,Y,1,NaN,NaN,NaN,Repairs,Refused,-781,Cash through the bank,HC,NaN,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN
5,1383531,199383,Cash loans,23703.930,315000.0,340573.5,NaN,315000.0,SATURDAY,8,Y,1,NaN,NaN,NaN,Everyday expenses,Approved,-684,Cash through the bank,XAP,Family,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,18.0,low_normal,Cash X-Sell: low,365243.0,-654.0,-144.0,-144.0,-137.0,1.0
6,2315218,175704,Cash loans,NaN,0.0,0.0,NaN,NaN,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Canceled,-14,XNA,XAP,NaN,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN
7,1656711,296299,Cash loans,NaN,0.0,0.0,NaN,NaN,MONDAY,7,Y,1,NaN,NaN,NaN,XNA,Canceled,-21,XNA,XAP,NaN,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN
8,2367563,342292,Cash loans,NaN,0.0,0.0,NaN,NaN,MONDAY,15,Y,1,NaN,NaN,NaN,XNA,Canceled,-386,XNA,XAP,NaN,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN
9,2579447,334349,Cash loans,NaN,0.0,0.0,NaN,NaN,SATURDAY,15,Y,1,NaN,NaN,NaN,XNA,Canceled,-57,XNA,XAP,NaN,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
previous_application[previous_application['AMT_CREDIT'].isnull()]
## NA means canceled, 0 usually means approved


,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
1127152,2204450,438387,Revolving loans,0.0,0.0,NaN,NaN,NaN,FRIDAY,10,Y,1,NaN,NaN,NaN,XAP,Approved,-608,XNA,XAP,NaN,Repeater,XNA,Cards,walk-in,Country-wide,20,Connectivity,0.0,XNA,Card Street,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
previous_application[(previous_application['AMT_DOWN_PAYMENT']==0)]

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.0,0.182832,0.867336,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
13,1397919,321676,Consumer loans,7654.860,53779.5,57564.0,0.0,53779.5,SUNDAY,15,Y,1,0.0,NaN,NaN,XAP,Approved,-408,Cash through the bank,XAP,Unaccompanied,New,Consumer Electronics,POS,XNA,Country-wide,200,Consumer electronics,8.0,low_action,POS household without interest,365243.0,-378.0,-168.0,-168.0,-163.0,1.0
14,2273188,270658,Consumer loans,9644.220,26550.0,27252.0,0.0,26550.0,SATURDAY,10,Y,1,0.0,NaN,NaN,XAP,Approved,-726,Cash through the bank,XAP,NaN,New,Construction Materials,POS,XNA,Stone,83,Consumer electronics,3.0,middle,POS household with interest,365243.0,-693.0,-633.0,-633.0,-627.0,0.0
27,1676258,433469,Cash loans,22242.825,247500.0,268083.0,0.0,247500.0,THURSDAY,14,Y,1,0.0,NaN,NaN,XNA,Approved,-1883,XNA,XAP,NaN,Repeater,XNA,Cash,x-sell,Country-wide,-1,Consumer electronics,18.0,high,Cash X-Sell: high,365243.0,-1853.0,-1343.0,-1343.0,-1334.0,1.0
30,2138466,170289,Consumer loans,3286.485,32562.0,32562.0,0.0,32562.0,FRIDAY,12,Y,1,0.0,0.196914,0.867336,XAP,Approved,-307,Cash through the bank,XAP,NaN,Refreshed,Photo / Cinema Equipment,POS,XNA,Country-wide,20,Connectivity,12.0,middle,POS mobile with interest,365243.0,-273.0,57.0,-183.0,-179.0,0.0
31,2191093,182450,Consumer loans,9789.255,100485.0,100485.0,0.0,100485.0,WEDNESDAY,9,Y,1,0.0,NaN,NaN,XAP,Approved,-391,Cash through the bank,XAP,NaN,Refreshed,Computers,POS,XNA,Stone,200,Consumer electronics,12.0,low_normal,POS household with interest,365243.0,-360.0,-30.0,-270.0,-261.0,0.0
35,2027074,208000,Consumer loans,12065.535,56655.0,66411.0,0.0,56655.0,SATURDAY,7,Y,1,0.0,NaN,NaN,XAP,Approved,-676,Cash through the bank,XAP,Unaccompanied,Repeater,Audio/Video,POS,XNA,Country-wide,50,Consumer electronics,6.0,middle,POS household with interest,365243.0,-645.0,-495.0,-495.0,-493.0,0.0
36,2766278,178375,Consumer loans,7511.085,32490.0,38380.5,0.0,32490.0,MONDAY,5,Y,1,0.0,NaN,NaN,XAP,Refused,-801,Cash through the bank,LIMIT,"Spouse, partner",Repeater,Consumer Electronics,POS,XNA,Country-wide,50,Consumer electronics,6.0,high,POS household with interest,NaN,NaN,NaN,NaN,NaN,NaN
38,1692577,125157,Consumer loans,9946.485,99319.5,109809.0,0.0,99319.5,SATURDAY,9,Y,1,0.0,NaN,NaN,XAP,Approved,-392,XNA,XAP,Unaccompanied,Repeater,Audio/Video,POS,XNA,Regional / Local,136,Consumer electronics,12.0,low_action,POS household without interest,365243.0,-362.0,-32.0,-242.0,-237.0,1.0
40,2317570,258076,Consumer loans,4539.285,18360.0,21834.0,0.0,18360.0,WEDNESDAY,10,Y,1,0.0,NaN,NaN,XAP,Approved,-795,Cash through the bank,XAP,Unaccompanied,Repeater,Mobile,POS,XNA,Country-wide,40,Connectivity,6.0,high,POS mobile with interest,365243.0,-764.0,-614.0,-614.0,-609.0,0.0


In [37]:
sum(previous_application['AMT_APPLICATION']*(previous_application['AMT_APPLICATION']/previous_application['AMT_ANNUITY']))/sum(previous_application['AMT_APPLICATION'])

nan

In [ ]:
def pa_f(c):
    # cannot run weighted avg ,kernel always exit
    length=c.shape[0]
    
    return(pd.Series({'# of NA annuity':sum(c['AMT_ANNUITY'].isnull()),
                     '% of NA annuity':sum(c['AMT_ANNUITY'].isnull())/length,
                      '# of 0 annuity':sum(c['AMT_ANNUITY']==0),
                     '% of NA annuity':sum(c['AMT_ANNUITY']==0)/length,
                       'avg of loan terms':(c['AMT_CREDIT']/c['ANNUITY']).mean(),   already have a variable in dataset
                      '% of >0 downpayment':sum(c['AMT_DOWN_PAYMENT']>0)/length,
                      '% of >0 goods price':sum(c['AMT_GOODS_PRICE']>0)/length,
                      '% of NA terms':sum(c['CNT_PAYMENT'].isnull())/length 
                     }))

In [4]:
def cal_term(col):
    col.fillna(0,inplace=True)
    col=col[col['AMT_ANNUITY']!=0]
    sum(col['AMT_APPLICATION']*(col['AMT_APPLICATION']/col['AMT_ANNUITY']))/sum(col['AMT_APPLICATION'])

In [ ]:
previous_application.groupby('SK_ID_CURR').apply(lambda g:cal_term(g))

In [ ]:
def previous_application(num_rows = None, nan_as_category = True):
    gc.diable()
    pa = pd.read_csv(r'D:\kaggle\home credit risk\DATA\previous_application.csv', nrows = num_rows)
    pa['AMT_GOODS_PRICE']=pa['AMT_GOODS_PRICE'].apply(lambda g:0 if g<0)
    pa['SELLERPLACE_AREA'][~(pa['SELLERPLACE_AREA']>0)]=0
    pa['DAYS_FIRST_DRAWING'][pa['DAYS_FIRST_DRAWING']==365243]=np.NaN
    pa['DAYS_LAST_DUE'][pa['DAYS_LAST_DUE']==365243]=np.NaN
    pa['DAYS_TERMINATION'][pa['DAYS_TERMINATION']==365243]=np.NaN
    
    pa.drop(['SK_ID_PREV'], axis= 1, inplace = True)
    
    
    
    
    
    
    
    
    
    
    
    
    #custome aggregations
    g=pa.groupby(['SK_ID_CURR']).apply(lambda col:pa_f(col))
    
    
    
    
    
    
    pa, cat_cols = one_hot_encoder(pa, nan_as_category= True)
    # General aggregations
    
    # delete rows that amount of annuity ==0
    pa.drop(pa[pa['AMT_ANNUITY'].isnull()].index,inplace=True)
    
    
    
    
    
    
    
    
    
    pa_agg = pa.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])
    pa_agg.columns = pd.Index(['PA_' + e[0] + "_" + e[1].upper() for e in pa_agg.columns.tolist()])
    # Count credit card lines
    pa_agg['PA_COUNT'] = pa.groupby('SK_ID_CURR').size()
    pa_agg=pd.merge(pa_agg,g,on=['SK_ID_CURR'])
    gc.enable()
    del pa,g
    gc.collect()
    return pa_agg

1.0

In [3]:
pos = pd.read_csv(r'D:\kaggle\home credit risk\DATA\POS_CASH_balance.csv')

In [6]:
pos

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0
5,2207092,342166,-32,12.0,12.0,Active,0,0
6,1110516,204376,-38,48.0,43.0,Active,0,0
7,1387235,153211,-35,36.0,36.0,Active,0,0
8,1220500,112740,-31,12.0,12.0,Active,0,0
9,2371489,274851,-32,24.0,16.0,Active,0,0


In [7]:
pos.sort_values(by=['SK_ID_CURR','SK_ID_PREV','MONTHS_BALANCE'])


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
7167007,1369693,100001,-57,4.0,4.0,Active,0,0
8789081,1369693,100001,-56,4.0,3.0,Active,0,0
7823681,1369693,100001,-55,4.0,2.0,Active,0,0
4704415,1369693,100001,-54,4.0,1.0,Active,0,0
2197888,1369693,100001,-53,4.0,0.0,Completed,0,0
1261679,1851984,100001,-96,4.0,2.0,Active,0,0
1891462,1851984,100001,-95,4.0,1.0,Active,7,7
8531326,1851984,100001,-94,4.0,0.0,Active,0,0
4928574,1851984,100001,-93,4.0,0.0,Completed,0,0
8473918,1038818,100002,-19,24.0,24.0,Active,0,0


whether the term of installment will change

In [8]:
pos['NAME_CONTRACT_STATUS'].value_counts()

Active                   9151119
Completed                 744883
Signed                     87260
Demand                      7065
Returned to the store       5461
Approved                    4917
Amortized debt               636
Canceled                      15
XNA                            2
Name: NAME_CONTRACT_STATUS, dtype: int64